<br/>

# Intro to Flux.jl

In the previous course, we learned how machine learning allows us to classify data as apples or bananas with a single neuron. However, some of those details are pretty fiddly! Fortunately, Julia has a powerful package that does much of the heavy lifting for us, called [`Flux.jl`](https://fluxml.github.io/).

*Using `Flux` will make classifying data and images much easier!*

## Using `Flux.jl`

We can get started with `Flux.jl` via:

In [1]:
# using Pkg; Pkg.add(["Flux", "Plots"])
using Flux, Plots

┌ Info: Recompiling stale cache file /home/olszewskip/.julia/compiled/v1.1/Plots/ld3vC.ji for Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1184


#### Helpful built-in functions

When working we'll `Flux`, we'll make use of built-in functionality that we've had to create for ourselves in previous notebooks.

For example, the sigmoid function, σ, that we have been using already lives within `Flux`:

In [3]:
?σ

"σ" can be typed by \sigma<tab>

search: σ logσ



```
σ(x) = 1 / (1 + exp(-x))
```

Classic [sigmoid](https://en.wikipedia.org/wiki/Sigmoid_function) activation function.


In [5]:
σ(0)

0.5

In [4]:
plot(σ, -5, 5, label="\\sigma", xlabel="x", ylabel="\\sigma\\(x\\)")

MethodError: MethodError: no method matching Float64(::Array{Float64,1})
Closest candidates are:
  Float64(!Matched::Int8) at float.jl:60
  Float64(!Matched::Int16) at float.jl:60
  Float64(!Matched::Int32) at float.jl:60
  ...

Importantly, `Flux` allows us to *automatically create neurons* with the **`Dense`** function. For example, in the last notebook, we were looking at a neuron with 2 inputs and 1 output:

 <img src="data/single-neuron.png" alt="Drawing" style="width: 500px;"/>

 We could create a neuron with two inputs and one output via

In [9]:
model = Dense(2, 1, σ)

Dense(2, 1, NNlib.σ)

This `model` object comes with places to store weights and biases:

In [10]:
model.W

Tracked 1×2 Array{Float32,2}:
 -1.21282  -1.33756

In [11]:
model.b

Tracked 1-element Array{Float32,1}:
 0.0f0

In [12]:
typeof(model.W)

TrackedArray{…,Array{Float32,2}}

In [13]:
x = rand(2)
model(x)

Tracked 1-element Array{Float32,1}:
 0.12760182f0

In [15]:
σ.(model.W * x + model.b)

Tracked 1-element Array{Float64,1}:
 0.12760182480567697

Unlike in previous notebooks, note that `W` is no longer a `Vector` (1D `Array`) and `b` is no longer a number! Both are now stored in so-called `TrackedArray`s and `W` is effectively being treated as a matrix with a single row. We'll see why below.

Other helpful built-in functionality includes ways to automatically calculate gradients and also the cost function that we've used in the previous course -

$$L(w, b) = \sum_i \left[y_i - f(x_i, w, b) \right]^2$$

If you normalize by dividing by the total number of elements, this becomes the "mean square error" function, which in `Flux` is named **`Flux.mse`**.

In [ ]:
methods(Flux.mse)

### Bringing it all together

Load the datasets that contain the features of the apple and banana images.

In [ ]:
using CSV, DataFrames

apples = DataFrame(CSV.File("data/apples.dat", delim='\t', allowmissing=:none, normalizenames=true))
bananas = DataFrame(CSV.File("data/bananas.dat", delim='\t', allowmissing=:none, normalizenames=true));

In [ ]:
x_apples  = [ [row.red, row.green] for row in eachrow(apples)]
x_bananas = [ [row.red, row.green] for row in eachrow(bananas)];

Concatenate the x (features) together to create a vector of all our datapoints, and create the corresponding vector of known labels:

In [ ]:
xs = [x_apples; x_bananas]
ys = [fill(0, size(x_apples)); fill(1, size(x_bananas))];

In [ ]:
model = Dense(2, 1, σ)

We can evaluate the model (currently initialized with random weights) to see what the output value is for a given input:

In [ ]:
model(xs[1])

And of course we can examine the current loss value for that datapoint:

In [ ]:
loss = Flux.mse(model(xs[1]), ys[1])

In [ ]:
typeof(loss)

### Backpropagation

In [ ]:
model.W

In [ ]:
model.W.grad

In [ ]:
using Flux.Tracker
back!(loss)

In [ ]:
model.W.grad

Now we have all the tools necessary to build a simple gradient descent algorithm!

### The easy way

You don't want to manually write out gradient descent algorithms every time! Flux, of course, also brings in lots of optimizers that can do this all for you.

In [ ]:
?SGD

In [ ]:
?Flux.train!

So we can simply define our loss function, an optimizer, and then call `train!`. That's basic machine learning with Flux.jl.

In [ ]:
model = Dense(2, 1, σ)
L(x,y) = Flux.mse(model(x), y)
opt = SGD(params(model))
Flux.train!(L, zip(xs, ys), opt)

## Visualize the result

In [ ]:
contour(0:.1:1, 0:.1:1, (x, y) -> model([x,y])[].data, fill=true)
scatter!(first.(x_apples), last.(x_apples), label="apples")
scatter!(first.(x_bananas), last.(x_bananas), label="bananas")
xlabel!("mean red value")
ylabel!("mean green value")